# 02. Unsupervised learning & clustering

#### Note: I'm so sorry this turned out to be really long.

In [ ]:
import os
import numpy as np
import pandas as pd
from tabulate import tabulate

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from scipy.interpolate import interp1d

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

In [ ]:
import datetime


import matplotlib.ticker as mticker
import matplotlib.gridspec as gridspec

import plotly.express as px



In [ ]:
data = '/home/masterdesky/data/world-bank'
out = './out/'

# Bold print for Jupyter Notebook
b1 = '\033[1m'
b0 = '\033[0m'

### Just some matplotlib and seaborn parameter tuning

In [ ]:
axistitlesize = 20
axisticksize = 17
axislabelsize = 26
axislegendsize = 23
axistextsize = 20
axiscbarfontsize = 15

# Set axtick dimensions
major_size = 6
major_width = 1.2
minor_size = 3
minor_width = 1
mpl.rcParams['xtick.major.size'] = major_size
mpl.rcParams['xtick.major.width'] = major_width
mpl.rcParams['xtick.minor.size'] = minor_size
mpl.rcParams['xtick.minor.width'] = minor_width
mpl.rcParams['ytick.major.size'] = major_size
mpl.rcParams['ytick.major.width'] = major_width
mpl.rcParams['ytick.minor.size'] = minor_size
mpl.rcParams['ytick.minor.width'] = minor_width

## 1. Reading data
The woldbank_development_2015.csv (can be found in the same folder with this notebook) file contains the World Development Indicators for the 2015 year, downloaded from [The World Bank's webpage](https://databank.worldbank.org/source/world-development-indicators#).


 - Look at the data in any text editor. Build up an overall sense how the data is built up and how is the missing values are represented.
 - Read the file into a pandas dataframe and tell pandas which special pattern means if a value is missing. 
 - The data is in a long format. Convert it into a wide format, where each row is a single country and the columns are the measured features, the Series Codes (eg the first column is 'EG.CFT.ACCS.ZS', the second is 'EG.ELC.ACCS.ZS'. Order of the columns does not matter)!
 - Keep only those rows, which represents counties and NOT regions. Luckily they are well separated in the order they occur!
 - Convert the features to numeric format, which will be needed for modeling!

### 1./a. Load in dataset

In [ ]:
os.listdir(data)

In [ ]:
Y = 2021
df = pd.read_csv(os.path.join(data, f'{Y}/world-bank-{Y}.csv'), sep=',')

In [ ]:
display(df.head(n=10))
display(df.tail(n=10))

The last two columns in the dataset are just appended information to the table, and could be left out from the analysis. The three lines preceding them are simply separating these two lines from the actual dataset, so they could be also left out. In summary, we can safely cut off the last 5 rows from the end of the table as they are not part of the data.

In [ ]:
df = df.iloc[:-5]

### 1./b. Explore columns and NaN values in the dataset

#### Number of missing values

In [ ]:
# Create a mask to analyze missing entries easier
nan_mask = df.isna()

In [ ]:
nan_count = nan_mask.sum()

In [ ]:
print('Count of missing values:\n' +
      '========================')
print(tabulate([[c, nan_count[c]] for c in nan_count.index], headers=['Feature', 'Count of NaNs']))

#### Corrected number of missing values

Very important to notice, there are lot of entries in the column `2015 [YR2015]`, which are simply denoted by `..`. These should be considered to be NaN values, but the `pandas.DataFrame.isna()` method won't detect them as such. We need to delete these manually to "replace" them with "true" NaN values.

In [ ]:
# Replace '..' with NaN
df = df.replace({'..' : float('nan')})

In [ ]:
display(df.head(n=10))
display(df.tail(n=10))

In [ ]:
# Create a mask to analyze missing entries easier
nan_mask = df.isna()

In [ ]:
nan_count = nan_mask.sum()

In [ ]:
print('Count of missing values:\n' +
      '========================')
print(tabulate([[c, nan_count[c]] for c in nan_count.index], headers=['Feature', 'Count of NaNs']))

### 1./c. Drop regions

In [ ]:
# Collect name of countries/regions in order of their occurences in the DataFrame
countries_regions = []
for c in df['Country Name']:
    if c not in countries_regions:
        countries_regions.append(c)

If we look at the order of countries in the `countries_regions` list we see, that countries in alphabetic order are folloed by regions in also alphabetic order. Since Zimbabwe is the last country in the alphabet by its name, we should drop every row in the DataFrame, which come after the last entry of `Zimbabwe`.

In [ ]:
# Find Zimbabwe in the `countries_regions` list and drop everything after that
countries = countries_regions[:countries_regions.index('Zimbabwe') + 1]
# Drop region data from the DataFrame
df_c = df[df['Country Name'].isin(countries)]
print('Dropped number of regions : {0}'.format(len(countries_regions) - len(countries)))
print('Dropped number of rows : {0}'.format(df.shape[0] - df_c.shape[0]))

### 1./d. Convert long to wide format

Since we truncate the dataset by fitting countries into a single line, we have to drop a column with unique entries. Why? The new wide format would look like similar to this:

| Index    | Country Name | Country Code | EG.CFT.ACCS.ZS | EG.ELC.ACCS.ZS | $\dots$  |
|:--------:|:------------:|:------------:|:--------------:|:--------------:|:--------:|
| 0        | Afghanistan  | AFG          | $30.1$         | $71.5$         | $\dots$  |
| 1        | Albania      | ALB          | $75.37$        | $100$          | $\dots$  |
| 2        | Algeria      | DZA          | $92.7$         | $99.931...$    | $\dots$  |
| $\vdots$ | $\vdots$     | $\vdots$     | $\vdots$       | $\vdots$       | $\ddots$ |

This format is however missing the `Series Name` column. To be able to clearly handle the description of each series by attaching it to the series itself in the table, we would require another, a third dimension. Because this is not feasible within the framework of `pandas`, I'm leaving this column out of the dataset. I create another dictionary however, to store this information for later use. Also I might use `Country Name` and the index column, and leave out the `Country Code`s

#### Occurence of series codes

In [ ]:
# Collect unique series codes and the number of their occurence in the dataset
sc = np.unique(df_c['Series Code'], return_counts=True)
# Collect the unique occurence counts of series codes in the dataset
# Ideally all series codes should exists for each country, thus this
# function should return with only one unique value
sc_c = np.unique(sc[1], return_counts=True)
print('There are {0} different features for each country.'.format(len(sc[0])))
print('There are {0} different countries exist in the dataset'.format(np.sum(sc_c[0])))
if len(sc_c[0]) == 1:
    print('All features exist for each country (however their values still could be NaN).')
else:
    print('There are features, which only exist for some countries!')

Since all series codes/features have an exsting entry in the dataset for each country, we can proceed to the next step of the current task.

#### Construct the new wide format

In [ ]:
# First save the descriptions of each series codes
features = {k : v for (k, v) in zip(df_c['Series Code'], df_c['Series Name'])}

In [ ]:
# A very simple pivot would convert the table to the desirable wide format
df_wide = df_c.pivot(index='Country Name', columns='Series Code', values=f'{Y} [YR{Y}]')

In [ ]:
df_wide

### 1./d. Convert features to numeric format

I don't really understand what does "coverting features to numeric format" would like to mean here. I'll simply convert all entries/**feature values** into numeric format. Let us first count the number of entries in the data table, which isn't in a numeric format, but in eg. `str`. It could be easily seen, that the number of non-float values are equal to the number of string values in the dataset. This trivially means, all non-numeric entries are indeed simple Python `str`s.

In [ ]:
def count_non_float_in_df(df):
    # Convert dataframe into an array of dimension (MxN),
    # where M is the number of features, while N is the number of countries
    df_arr = np.array(df).T
    # Create an array to store str() counts for all feature columns
    df_str_c = np.zeros(df_arr.shape[0])

    # Count number of string values in each feature columns
    for i, f in enumerate(df_arr):
        df_str_c[i] = sum(1 for v in f if type(v) == str)

    return df_str_c

In [ ]:
df_wide_str_c = count_non_float_in_df(df=df_wide)

print('Total number of non-float feature values in the dataset : {0}'.format(int(sum(df_wide_str_c))))

In [ ]:
# Convert all columns to numeric format
for c in df_wide.columns:
    # Convert column 'c' of the DataFrame to numeric format
    df_wide[c] = pd.to_numeric(df_wide[c])

In [ ]:
df_wide_str_c = count_non_float_in_df(df=df_wide)

print('Total number of non-float feature values in the dataset (after conversion, should be 0 if successful) : {0}'.format(int(sum(df_wide_str_c))))

## 2. Data preprocessing and inspection

 - Visualize the missing values!
 - Keep only those countries which has less than 700 missing features in the original table and keep features which were missing in less than 20 country in the original table
 - Visualize the missing values again! Now really only a few entries are missing. Impute the missing values with its feature's mean value!
 - How many counties and features do we have left?
 - Read the kept features' descriptions. In the original table the Series Name describe the meaning of the features. What do you think, based only on these information, which counties are the most similar to Hungary? And Norway?  

### 2./a. Visualize the missing values

In [ ]:
fig, axes = plt.subplots(figsize=(40,40))
axes.set_aspect('equal')

axes.imshow(np.array(df_wide.isna()))

# Remove XY ticks for now
axes.set_xticks([])
axes.set_yticks([])

axes.set_title('Fig. 1. Missing entries in the original wide-format dataset',
               fontsize=axistitlesize+15, y=-0.28)
# Lapelpad needed because of the removed ticks
lp = 20
axes.set_xlabel('Columns', fontsize=axislabelsize+15, fontweight='bold', labelpad=lp)
axes.set_ylabel('Rows', fontsize=axislabelsize+15, fontweight='bold', labelpad=lp)

plt.show()

This in itself is a truly mesmerizing and interesting picture, which I'll definitely set as a cover photo somewhere, but we can also observe a number of important and meaningful things about the dataset on it. It seem so there are columns, which do not store any information whatsover because they're completely filled with NaN entries. I will live with the assumption for now that the "missingness" of entires itself do not contain any useful information for this homework, and I'll simply delete them from the dataset.

In [ ]:
# Drop only those columns, which have absolutely none number values
df_wide_dn = df_wide.dropna(axis='columns', thresh=1)
print('Number of dropped columns : {0}'.format(df_wide.columns.size - df_wide_dn.columns.size))
print('Number of remaining columns : {0}'.format(df_wide_dn.columns.size))

In [ ]:
df_wide_dn

In [ ]:
fig, axes = plt.subplots(figsize=(40,40))
axes.set_aspect('equal')

axes.imshow(np.array(df_wide_dn.isna()))

# Remove XY ticks for now
axes.set_xticks([])
axes.set_yticks([])

axes.set_title('Fig. 2. Missing entries in the wide-format dataset without the full NaN columns',
               fontsize=axistitlesize+15, y=-0.28)
# Lapelpad needed because of the removed ticks
lp = 20
axes.set_xlabel('Columns', fontsize=axislabelsize+15, fontweight='bold', labelpad=lp)
axes.set_ylabel('Rows', fontsize=axislabelsize+15, fontweight='bold', labelpad=lp)

plt.show()

### 2./b. Drop out countries with lot of missing values

Since there are $1437$ series as we've seen in the first task, we need to drop countries with at least $737$ existing entries.

In [ ]:
# Drop only those columns, which have absolutely none number values
df_w = df_wide_dn.dropna(axis='index', thresh=346)
print('Number of dropped countries : {0}'.format(df_wide_dn.index.size - df_w.index.size))
print('Number of remaining countries : {0}'.format(df_w.index.size))

In [ ]:
fig, axes = plt.subplots(figsize=(40,40))
axes.set_aspect('equal')

axes.imshow(np.array(df_w.isna()))

# Remove XY ticks for now
axes.set_xticks([])
axes.set_yticks([])

axes.set_title('Fig. 3. Missing entries in the wide-format dataset without the full NaN columns and dropped out countries',
               fontsize=axistitlesize+15, y=-0.32)
# Lapelpad needed because of the removed ticks
lp = 20
axes.set_xlabel('Columns', fontsize=axislabelsize+15, fontweight='bold', labelpad=lp)
axes.set_ylabel('Rows', fontsize=axislabelsize+15, fontweight='bold', labelpad=lp)

plt.show()

### 2./c. Fill NaN entries


Filling all missings entries by the mean of its feature values as asked in the description.

In [ ]:
# NOTE: The new variable has an extra `f` at its end
df_wf = df_w.fillna(df_w.mean())

In [ ]:
# Selected final list of countries
countries_sc = list(df_wf.index)

### 2./d. Which country is the closest to Hungary? And Norway?

Based on the current dataset `df_wf` it is not completely trivial to answer this question. For example we can define a score, which could measure the "distance" of each country from Hungary and Norway. This score could be interpeted as some function of the differences between features of two separate countries. In this framework we should find the country corresponding to the global extrema of this scoring function (eg. a minima in the case of the distance interpretation). I'll simply use the MSE of features compared to Hungary/Norway as a scoring function for this sub-task.

#### Create anf calculate score for a country

In [ ]:
MSE = {}

In [ ]:
def add_country_2_MSE(MSE, ref):

    # Calculate MSE values
    mse_list = []
    for c in df_wf.index:
        mse_list.append(mean_squared_error(df_wf.loc[ref], df_wf.loc[c]))
    # Normalize dataset
    mse_list = np.array(mse_list) / np.max(mse_list)
    MSE[ref] = {c : v for (c, v) in zip(df_wf.index, mse_list)}

    return MSE

In [ ]:
MSE = add_country_2_MSE(MSE, ref='Hungary')
MSE = add_country_2_MSE(MSE, ref='Norway')

#### Visualize the scores for Hungary

In [ ]:
def plot_MSE_country(country):

    # Sort countries by score values
    x = list(df_wf.index)
    y = list(MSE[country].values())
    y, x = (list(z) for z in zip(*sorted(zip(y, x))))

    # Documentation was very useful @:
    # https://plotly.com/python/bar-charts/
    fig = px.bar(y=x[::-1], x=y[::-1], text=y[::-1],
                 color=y[::-1],
                 log_x=True,
                 title='Fig. 4. MSE of features compared to Hungary',
                 labels={
                     'y': 'Countries in ascending order by score',
                     'x': 'Log of score values',
                 },
                 orientation='h', width=1300, height=2600)
    fig.update_traces(hovertemplate='Country : %{y}<br></br>MSE : %{x:.3e}',
                      texttemplate='%{text:.3e}',
                      textposition='outside')
    fig.update_layout(coloraxis_colorbar=dict(
                            dtick=0.05,
                            tickfont=dict(
                                family='Arial',
                                color='black',
                                size=22)
                            ),
                      font=dict(
                          family='Arial',
                          size=12,
                          color='black'
                            ),
                      uniformtext_minsize=4,
                      uniformtext_mode='hide'
                     )
    fig.show()

In [ ]:
plot_MSE_country(country='Hungary')

### Discussion

According to a simple MSE score, Kazahstan is the closest one to our country in economic aspects, which is a bit unsettling information. The closest country to Norway is Denmark, which is quite more comforting to Norway people. Extra info: I asked my dad, who's an expert and researcher in macroeconomics and he confirmed that Kazahstan is a pretty good tip and it is actually considered as one of closer countries to Hungary in many development indicators.

In [ ]:
def closest_country_by_MSE(ref):
    
    x = list(df_wf.index)
    y = list(MSE[ref].values())
    y, x = (list(z) for z in zip(*sorted(zip(y, x))))
    
    print('The "closest" country to [{0}] by this scoring function is [{1}].'.format(ref, x[::-1][-2]))

In [ ]:
closest_country_by_MSE(ref='Hungary')
closest_country_by_MSE(ref='Norway')

## 3. PCA
 - Perform PCA with 3 principal components on the filtered, imputed data (from now on, data refers to the filtered, imputed dataset)
 - Plot the three embedded 2D combination next to each other (0 vs 1, 0 vs 2 and 1 vs 2)
 - It seems that the embedding is really dominated by a single direction. Normalize the data (each feature should have zero mean and unit variance after normalization) and re-do the PCA and the plotting (do not delete the previous plots, just make new ones).

### 3./a. Calculate and visualize PCA

I actually salvaged these old PCA calc. and plotting functions of mine from one of my projects from last year.

In [ ]:
def scale_data(X):
    """
    Normalize the data to have zero mean and unit variance.
    """
    # Initialize
    scaler = StandardScaler()
    # Compute the mean and standard dev. and scale the dataset `X`
    X = pd.DataFrame(scaler.fit_transform(X), index=df_wf.index, columns=df_wf.columns)

    return X

In [ ]:
def PCA(X, n_components=3):
    
    # Permorm the PCA and return the newly created dataset
    pca = decomposition.PCA(n_components=n_components)
    pca.fit(X)
    X_pca = pca.transform(X)
    
    # Ex-variance to measure impact of PCA features
    ex_var = np.var(X_pca, axis=0)
    ex_var_ratio = ex_var/np.sum(ex_var)
    
    return pca, X_pca, ex_var_ratio

In [ ]:
def plot_pca_scatter(X_pca, input_title=None, zoom=10):
    
    ncols = 3
    nrows = (len(perplexity)-1)//ncols + 1
    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*10, nrows*10))
    
    # Set a colormap for scatter points
    # Color points by countries in alphabetic order
    colors = cm.viridis(np.linspace(0, 1, num=len(X_pca)))
    alpha = 0.6
    
    # Which pair of PCs plotted in different columns on the plot
    pairs = [(0, 1), (1, 2), (0, 2)]
    labels = [('First', 'Second'), ('Second', 'Third'), ('First', 'Third')]

    for j in range(nrows):
        for i in range(ncols):
            ax = axes[j][i]
            
            X = X_pca[:, pairs[i][0]]
            Y = X_pca[:, pairs[i][1]]
            ax.scatter(X, Y,
                       color=colors, alpha=alpha)
            
            # Some string magic to hide offset text and write scale on the axis labels
            # 1. Turn off offset
            plt.setp(ax.get_xaxis().get_offset_text(), visible=False)
            plt.setp(ax.get_yaxis().get_offset_text(), visible=False)
            # 2. Get scale of axes
            x_lim = ax.get_xlim()
            y_lim = ax.get_ylim()
            # 3. Select greater limit in magnitude, force it to scientific notation
            # and trim down the exponent
            x_max = np.max(np.abs(x_lim))
            y_max = np.max(np.abs(y_lim))
            xy_max = np.max((x_max, y_max))
            
            # Do nothing special in the first row
            if j == 0:
                x_offset = int(('%E' % x_max).split('E')[-1])
                y_offset = int(('%E' % y_max).split('E')[-1])
            # Zoom on the center of the dataset in the second row
            elif j == 1:
                c = (np.mean(X), np.mean(Y))
                ax.set_xlim(c[0] - xy_max/zoom, c[0] + xy_max/zoom)
                ax.set_ylim(c[1] - xy_max/zoom, c[1] + xy_max/zoom)
                # Scale offsets accordingly to zooming/scalong factor
                x_offset = int(('%E' % xy_max).split('E')[-1]) - int(np.log10(zoom))
                y_offset = int(('%E' % xy_max).split('E')[-1]) - int(np.log10(zoom))
            
            # Format labels and ticks
            ax.set_xlabel('{0} PC [$\\times 10^{{{1}}}$]'.format(labels[i][0], x_offset),
                               fontsize=axislabelsize+6, fontweight='bold')
            ax.set_ylabel('{0} PC [$\\times 10^{{{1}}}$]'.format(labels[i][1], y_offset),
                               fontsize=axislabelsize+6, fontweight='bold')
            ax.tick_params(axis='both', which='major', labelsize=axisticksize+6)
    
    fig.suptitle(input_title,
                 fontsize=axistitlesize+6, y=0.04)
    
    plt.show()

In [ ]:
def get_impactful_features(pca, features, get_n=8):
    
    pca_basis = pca.components_
    
    pc_f_idx = []
    pc_f = []
    for pc in range(pca_basis.shape[0]):
        pc_current = pca_basis[pc]
        pc_n_f_idx = []
        pc_n_f = []
        for i in range(get_n):
            f_idx = np.where(np.abs(pc_current) == sorted(np.abs(pc_current))[::-1][i])[0][0]
            pc_n_f_idx.append(f_idx)
            pc_n_f.append(list(features.keys())[f_idx])
        pc_f_idx.append(pc_n_f_idx)
        pc_f.append(pc_n_f)

    return pc_f_idx, pc_f

In [ ]:
def pca_components(pca, pc_f_idx, pc_f,
                   input_title=None):
    
    nrows = 1
    ncols = 3
    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*16, nrows*10))
    fig.subplots_adjust(wspace=0.3)
    
    pca_basis = pca.components_
    for i in range(ncols):
        ax = axes[i]
        x = pc_f[i]
        y = pca_basis[i][pc_f_idx[i]]
        
        # Set a colormap for bars
        # Color bars by their value using a diverging cmap
        y_norm = np.array([e/np.abs(np.min(y)) if e < 0 else e/np.abs(np.max(y)) for e in y])
        m = interp1d([-1, 1], [0, 1])
        colors = cm.RdBu_r(m(y_norm))
        ax.barh(x, y,
                color=colors, align='center')
        
        ax.set_title('PC$_{{{0}}}$ components'.format(i+1), fontsize=axistitlesize, fontweight='bold')
        ax.set_xlabel('Impact of features', fontsize=axislabelsize, fontweight='bold')
        ax.tick_params(axis='both', which='major', labelsize=axisticksize)
    
    fig.suptitle(input_title,
                 fontsize=axistitlesize+25, y=0.0)
    
    plt.show()

#### Unscaled PCA

In [ ]:
pca, X_pca, ex_var_ratio = PCA(X=df_wf, n_components=3)
print(ex_var_ratio)
print('Weight if first three PC : {0:.3f}%'.format(ex_var_ratio[:3].sum()*100))

In [ ]:
input_title = 'Fig. 5. Feature map of the components outputted from the PCA of the dataset'
plot_pca_scatter(X_pca=X_pca, input_title=input_title, zoom=13)

In [ ]:
# Get the list of the N most impactful features in absolute values
pc_f_idx, pc_f = get_impactful_features(pca, features, get_n=8)

In [ ]:
pca_components(pca, pc_f_idx, pc_f,
               input_title='Fig. 6. Top features by their impact in absolute value on different PCs')

### 3./b. Normalize the dataset and redo the PCA

#### Scaled PCA

In [ ]:
# Scaled dataset in wide format, with already filled NaN values
df_wf_sc = scale_data(X=df_wf)

In [ ]:
pca, X_pca, ex_var_ratio = PCA(X=df_wf_sc, n_components=3)
print(ex_var_ratio)
print('Weight if first three PC : {0:.3f}%'.format(ex_var_ratio[:3].sum()*100))

In [ ]:
input_title = 'Fig. 7. Feature map of the components outputted from the PCA of the scaled and normalized dataset'
plot_pca_scatter(X_pca=X_pca, input_title=input_title, zoom=5)

In [ ]:
# Get the list of the N most impactful features in absolute values
pc_f_idx, pc_f = get_impactful_features(pca, features, get_n=8)

In [ ]:
pca_components(pca, pc_f_idx, pc_f,
               input_title='Fig. 8. Top features by their impact in absolute value on different PCs with the scaled dataset')

## 4. T-SNE
 - Perform T-SNE on the scaled data with 2 components
 - Plot the embeddings results. Add a text label for each point to make it possible to interpret the results. It will not be possible to read all, but try to make it useful, see the attached image as an example!
 - Highlight Hungary and Norway! Which countries are the closest one to Hungary and Norway? 

### 4./a. Perform and visualize t-SNE

In [ ]:
# Perform t-SNE in the scaled data with 2 components
df_embedded_variation = []
perplexity = [2, 5, 10, 30, 50, 100]
for p in perplexity:
    df_embedded_variation.append(TSNE(n_components=2, perplexity=p).fit_transform(df_wf_sc))

### Color countries in alphabetic order

In [ ]:
nrows = 2
ncols = 3
fig, axes = plt.subplots(nrows, ncols, figsize=(12*ncols, 12*nrows))

# scatter point radius
sc_rad = 9

# Set a colormap for scatter points
# Color points by countries in alphabetic order
colors = cm.viridis(np.linspace(0, 1, num=len(df_wf_sc)))
markup_colors = ['tab:red', 'tab:orange']

for i in range(nrows):
    for j in range(ncols):
        ax = axes[i][j]
        # Select embedding with the correct perplexity
        df_embedded = df_embedded_variation[i*ncols + j]
        x = df_embedded[:,0]
        y = df_embedded[:,1]
        ax.scatter(x, y,
                   color=colors, s=sc_rad**2, alpha=0.7)
        for c_idx, c in enumerate(['Hungary', 'Norway']):
            idx = list(df_wf_sc.index).index(c)
            ax.scatter(x[idx], y[idx], label=c,
                       color=markup_colors[c_idx], s=(sc_rad+10)**2, alpha=0.8)

        ax.set_title('Perplexity = {0}'.format(perplexity[i*ncols + j]), fontsize=axistitlesize, fontweight='bold')
        ax.set_xlabel('Embedding dim. 1', fontsize=axislabelsize, fontweight='bold', labelpad=15)
        ax.set_ylabel('Embedding dim. 2', fontsize=axislabelsize, fontweight='bold', labelpad=15)
        ax.tick_params(axis='both', which='major', labelsize=axisticksize)
        ax.legend(loc='best', fontsize=axislegendsize)

fig.suptitle('Fig. 9. Embeddings of a t-SNE performed with two components.\nCountries are colored in alphabetic order.',
             fontsize=axistitlesize+12, y=0.04)
        
plt.show()

### Color countries by their geographic position

I'll color the points corresponding to countries by their latitude coordinates. The reason for this, because there is well-known correlation of WDI and a country being on the northern/southern hemisphere. The reason for this correlation could be found in the arrangement of climate zones, which happens to be varying along the axis of latitude.

In [ ]:
import folium
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='tsne')

In [ ]:
lat = []
long = []
for c in list(df_wf_sc.index):
    loc = geolocator.geocode(c.split(',')[0])
    lat.append(loc.latitude)
    long.append(loc.longitude)

In [ ]:
ncols = 3
nrows = (len(perplexity)-1)//ncols + 1
fig, axes = plt.subplots(nrows, ncols, figsize=(12*ncols, 12*nrows))

# scatter point radius
sc_rad = 9

# Set a colormap for scatter points
# Color points by countries in alphabetic order
color_by = lat
coord_ref = np.max((np.abs(np.min(color_by)), np.abs(np.max(color_by))))
m = interp1d([-coord_ref, coord_ref], [0, 1])
colors = cm.seismic(m(color_by))

for i in range(nrows):
    for j in range(ncols):
        ax = axes[i][j]
        # Select embedding with the correct perplexity
        df_embedded = df_embedded_variation[i*ncols + j]
        x = df_embedded[:,0]
        y = df_embedded[:,1]
        ax.scatter(x, y,
                   color=colors, s=sc_rad**2, alpha=0.7)

        ax.set_title('Perplexity = {0}'.format(perplexity[i*ncols + j]), fontsize=axistitlesize, fontweight='bold')
        ax.set_xlabel('Embedding dim. 1', fontsize=axislabelsize, fontweight='bold', labelpad=15)
        ax.set_ylabel('Embedding dim. 2', fontsize=axislabelsize, fontweight='bold', labelpad=15)
        ax.tick_params(axis='both', which='major', labelsize=axisticksize)

# Thank you Stackoverflow
# https://stackoverflow.com/questions/43805821/matplotlib-add-colorbar-to-non-mappable-object
# https://stackoverflow.com/questions/15908371/matplotlib-colorbars-and-its-text-labels
# https://stackoverflow.com/questions/16702479/matplotlib-colorbar-placement-and-size
# https://stackoverflow.com/questions/33443334/how-to-decrease-colorbar-width-in-matplotlib
norm = mpl.colors.Normalize(vmin=int(-coord_ref)-1, vmax=int(coord_ref)+1)
sm = plt.cm.ScalarMappable(cmap=plt.cm.seismic, norm=norm)
sm.set_array([])
cbar_ticks = np.linspace(int(-coord_ref)-1, int(coord_ref)+1, 10, endpoint=True)
cbar = fig.colorbar(sm, ticks=cbar_ticks,
                    boundaries=np.arange(int(-coord_ref)-1, int(coord_ref)+1+0.01, 0.01),
                    pad=0.01, aspect=30,
                    ax=axes.ravel().tolist())
cbar.set_label('Latitudes [$\mathrm{arc\ degrees}$]', fontsize=axiscbarfontsize+20, labelpad=15, rotation=90)
cbar.ax.set_yticklabels(['{:.0f}'.format(x) for x in cbar_ticks], fontsize=axiscbarfontsize+10, weight='bold')


fig.suptitle('Fig. 10. Embeddings of a t-SNE performed with two components.\nCountries are colored by their latitude coordinates.',
             fontsize=axistitlesize+12, y=0.04)

plt.show()

### 4./b. Annotate figures with country names

Add a text label for each point to make it possible to interpret the results. It will not be possible to read all, but try to make it useful, see the attached image as an example!

**No.** This is just so bad.

In [ ]:
ncols = 3
nrows = (len(perplexity)-1)//ncols + 1
fig, axes = plt.subplots(nrows, ncols, figsize=(12*ncols, 12*nrows))

# scatter point radius
sc_rad = 9

# Set a colormap for scatter points
# Color points by countries in alphabetic order
colors = cm.viridis(np.linspace(0, 1, num=len(df_wf_sc)))
markup_colors = ['tab:red', 'tab:orange']

for i in range(nrows):
    for j in range(ncols):
        ax = axes[i][j]
        # Select embedding with the correct perplexity
        df_embedded = df_embedded_variation[i*ncols + j]
        x = df_embedded[:,0]
        y = df_embedded[:,1]
        ax.scatter(x, y,
                   color=colors, s=sc_rad**2, alpha=0.7)
        for c_idx, c in enumerate(['Hungary', 'Norway']):
            idx = list(df_wf_sc.index).index(c)
            ax.scatter(x[idx], y[idx], label=c,
                       color=markup_colors[c_idx], s=folium(sc_rad+10)**2, alpha=0.8)

        # Annotation
        for k, txt in enumerate(list(df_wf_sc.index)):
            ax.annotate(txt, (x[k], y[k]))
        
        ax.set_title('Perplexity = {0}'.format(perplexity[i*ncols + j]), fontsize=axistitlesize, fontweight='bold')
        ax.set_xlabel('Embedding dim. 1', fontsize=axislabelsize, fontweight='bold', labelpad=15)
        ax.set_ylabel('Embedding dim. 2', fontsize=axislabelsize, fontweight='bold', labelpad=15)
        ax.tick_params(axis='both', which='major', labelsize=axisticksize)
        ax.legend(loc='best', fontsize=axislegendsize)

fig.suptitle('Fig. 11. Embeddings of a t-SNE performed with two components with annotations.\nCountries are colored in alphabetic order.',
             fontsize=axistitlesize+12, y=0.04)
        
plt.show()

### 4./c. Countries closest to Hungary and Norway

We should find a distance measure to interpet the "closest country" to another country in the ensemble above. I'll simply use the `l1` and `l2` measures here to find the closest points.

In [ ]:
def ln(a, b, n):
    """
    Calculates the n-norm of two input vectors
    
    Parameters:
    -----------
    a, b : np.array
        The two arrays between the n-norm should be measured
    """
    return np.linalg.norm((a - b), ord=n)

In [ ]:
def distance_from_country(country):
    """
    Returns the l1 and l2 distances of all countries to a specific country.
    
    Parameters:
    -----------
    country : str
        The reference country's name in the dataset.
    
    Returns:
    --------
    distances : dict
        This dictionary would store all l1 and l2 distances in different embeddings ordered by countries as its keys.
        This would look something like this:
        ```python
        distances = {
            'Afghanistan' : [[emb_p_2_l1, emb_p_2_l2], [emb_p_5_l1, emb_p_5_l1], ... , [emb_p_100_l1, emb_p_100_l2]],
            'Albania'     : [ ... ],
            ...
        }
        ```
    """
    distances = {}
    # List of all countries
    countries = list(df_wf_sc.index)
    # Index of country
    ref_idx = countries.index(country)
    # Calculate l1 and l2 distances
    for c_idx, c in enumerate(countries):
        if c != country:
            distances[c] = []
            for emb in df_embedded_variation:
                a = emb[ref_idx]
                b = emb[c_idx]
                d_ln = []
                for n in range(1,3):
                    d_ln.append(ln(a, b, n))
                distances[c].append(d_ln)

    return distances

Choose some of the closest countries and plot their distance per t-SNE embeddings.

In [ ]:
def closest_countries_tsne(distances, top_n=8):
    """
    Collect the best N countries and their corresponding data from the different t-SNE runs.
    
    """
    # Reshape into shape (N, p_n, n_c)
    # Where
    #   `N` is number of different distance measures (currently l1 and l2)
    #   `p_n` is the number of different perplexity values used for t-SNE
    #   `c_n` is the number of countires in the `distances` dictionary
    arr = np.array(list(distances.values())).T
    
    # List of countries
    countries = list(distances.keys())
    
    best = []
    for lN in arr:
        best_N = []
        for emb in lN:
            # Sort countries by values in emb
            e, c = (list(z) for z in zip(*sorted(zip(emb, countries))))
            # Select the top n countries
            best_N.append([c[:top_n], e[:top_n]])
        best.append(best_N)
    
    return best

In [ ]:
def plot_closest_tsne(best,
                      input_title=None):
    """
    Visualize the output from `closest_countries_tsne()`
    
    Parameters:
    -----------
    best : list
        Who would have thought? This is the output from `closest_countries_tsne()`! Contains information
    """
    
    ncols = 3
    nrows = (len(perplexity)-1)//ncols + 1
    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*14, nrows*10))
    fig.subplots_adjust(hspace=0.5)
    
    colors = ['firebrick', 'cornflowerblue']
    
    for i in range(nrows):
        for j in range(ncols):
            ax = axes[i][j]
            x_ticks = []
            x_ticklabels = []
            for n in range(2):
                data = best[n][i*ncols + j]
                c = data[0]
                e = data[1]
                x = np.array([k for k in range(len(c))]) + 0.2 * (-1)**n
                
                ax.bar(x, e, label='L{n} distance'.format(n=n+1),
                       color=colors[n], width=0.4, align='center')

                x_ticks += list(x)
                x_ticklabels += c
            
            # Set xticks
            ax.set_xticks(x_ticks)
            ax.set_xticklabels(x_ticklabels)
        
            ax.set_title('Perplexity = {0}'.format(perplexity[i*ncols + j]), fontsize=axistitlesize, fontweight='bold')
            ax.set_ylabel('Distance to reference country', fontsize=axislabelsize, fontweight='bold', labelpad=15)
            ax.tick_params(axis='both', which='major', labelsize=axisticksize)
            ax.tick_params(axis='x', which='major', rotation=80)
            
            ax.legend(fontsize=axislegendsize)
    
    fig.suptitle(input_title,
                 fontsize=axistitlesize+12, y=-0.02)
    
    plt.show()

In [ ]:
distances_hu = distance_from_country(country='Hungary')
distances_nw = distance_from_country(country='Norway')
distances_ag = distance_from_country(country='Afghanistan')

In [ ]:
best_hu = closest_countries_tsne(distances=distances_hu, top_n=8)
best_nw = closest_countries_tsne(distances=distances_nw, top_n=8)
best_ag = closest_countries_tsne(distances=distances_ag, top_n=8)

In [ ]:
plot_closest_tsne(best=best_hu,
                  input_title='Fig. 12. The 8 closest countries to Hungary in the t-SNE analysis by L1 and L2 metrics')

In [ ]:
plot_closest_tsne(best=best_nw,
                  input_title='Fig. 13. The 8 closest countries to Norway in the t-SNE analysis by L1 and L2 metrics')

In [ ]:
plot_closest_tsne(best=best_ag,
                  input_title='Fig. 14. The 8 closest countries to Afghanistan in the t-SNE analysis by L1 and L2 metrics')

## 5. Hierarchical clustering

 - Perform hierarchical clustering on the filtered and scaled data (hint: use seaborn **(\*sklearn lol)**)
 - Try to plot in a way that all country's name is visible
 - Write down your impressions that you got from this plot!

In [ ]:
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

### 5./a. Perform clustering

For the clustering I've used the hierarchical agglomerative clustering with Ward's minimum variance method from `sklearn`. Ward's method is an agglomerative ("bottom to top up") type clustering, based on the minimalization of the Within Cluster Variance. To understand this, first a $\varrho$ euclidean distance should be defined between two individual points in the feature space:

$$
\varrho_{ij} = \varrho \left( X_{i}, X_{j} \right) = \left| \left| X_{i} - X_{j} \right| \right|
$$

With this definition, we can also define a $\sigma_{ij}^{2}$ variance or equivalently the $\varrho_{ij}^{2}$ squared euclidean distance (SED):

$$
\sigma_{ij}^{2} = \varrho_{ij}^{2} = \left| \left| X_{i} - X_{j} \right| \right|^{\ 2}
$$

At the beginning, we assign all individual feature vectors into a singleton (a cluster with a single element). Thus, at the first step for $N$ elements we have $N$ number of clusters. At the next step it is considered, that with the merge of two clusters $A$ and $B$, how much the total variance will be increased as follows:

$$
\Delta \left( A, B \right)
=
\sum_{i \in A \cup B} \left| \left| \vec{x}_{i} - \vec{c}_{A \cup B} \right| \right|^{\ 2}
-
\sum_{i \in A} \left| \left| \vec{x}_{i} - \vec{c}_{A} \right| \right|^{\ 2}
-
\sum_{i \in B} \left| \left| \vec{x}_{i} - \vec{c}_{B} \right| \right|^{\ 2}
$$

Here $\Delta \left( A, B \right)$ is the merging cost (variance growth) for merging clusters $A$ and $B$, $\vec{x}_{i}$ is the $i$th feature vector in the corresponding cluster and $\vec{c}$ is the centroid of the same cluster.  
Ward's method tries to keep this variance change as small as possible, creating $k$ clusters at the end, which $k$ parameter is predefined at the start of the clustering.

#### Sourced from:
- Joe H. Ward Jr. (1963) Hierarchical Grouping to Optimize an Objective Function, Journal of the American Statistical Association, 58:301, 236-244, DOI: [10.1080/01621459.1963.10500845](https://doi.org/10.1080/01621459.1963.10500845)
- http://scikit-learn.sourceforge.net/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html
- https://en.wikipedia.org/wiki/Ward's_method
- https://www.stat.cmu.edu/~cshalizi/350/lectures/08/lecture-08.pdf

In [ ]:
def get_centroids(data):
    """
    Gather the centroids of the created clusters from a datatable with additional `cluster_id` info.
    
    Parameters:
    -----------
    data : `pandas.DataFrame`
        desc
    
    Returns:
    --------
    centroids : `pandas.DataFrame`
        Centroids of the created clusters by features
    """
    indeces = list(set(data['cluster_id'].values))
    
    # Get the mean values of each feature inside the clusters and define these as centroids
    # For similar behaviour in eg. `sklearn`, see __doc__ for 'sklearn.cluster.KMeans().cluster_centers_':
    # (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)
    cd = {idx : [data[(data['cluster_id']==idx)].values.mean() \
               for feature in data.columns[:-1]] for idx in indeces}
    
    # Create dataframe to contain new centroids
    centroids = pd.DataFrame(data=cd)

    return centroids

In [ ]:
def perform_agglomerative(data, n_clusters=6, linkage='ward'):
    """
    Perform and hierarchical agglomerative clustering using `sklearn` with an arbitrary linking method
    
    Parameters:
    -----------
    data : `pandas.DataFrame`
        desc
    
    Returns:
    --------
    agglomerative : `sklearn.cluster._agglomerative.AgglomerativeClustering`
        The agglomerative clustering model fitted on the input data.
        
    df : `pandas.DataFrame`
        The input data with an additional column
    """
    df = data.copy()
    # Perform clustering
    agglomerative = AgglomerativeClustering(linkage=linkage, n_clusters=n_clusters)
    # Fit on the full dataset
    agglomerative.fit(df)
    # Predict on the original dataset
    df['cluster_id'] = agglomerative.labels_
    # Return the number of elements of clusters
    cluster_sizes = df['cluster_id'].value_counts()

    return agglomerative, df, cluster_sizes

In [ ]:
# Possible linkages: [ 'ward' | 'average' | 'complete' | 'single' ]
# Suggested: Ward's method ['ward']
linkage = 'ward'
n_clusters = 8

In [ ]:
agglomerative, df_clustering, cluster_sizes = perform_agglomerative(df_wf_sc, n_clusters=n_clusters, linkage=linkage)

# Reporting after agglomerative clustering
print(b1 + 'REPORT FOR THE SCALED, FILLED DATASET:' + b0)
clusters_indeces = list(set(df_clustering.cluster_id.values))
N_CLUSTERS = len(clusters_indeces)
centroids = get_centroids(df_clustering) 
print(b1 + 'Cluster centroids:' + b0 + '\n{0}\n'.format(centroids))
print((b1 + 'Total number of created clusters:' + b0 + ' {0}\n' + b0).format(N_CLUSTERS))
print(b1 + 'Cluster sizes:' + b0 + '\n{0}'.format(cluster_sizes))

### 5./b. Visualize clustering

Color the scattered points on the t-SNE graphs by clusters. Label the clusters on the graph also.

In [ ]:
ncols = 3
nrows = (len(perplexity)-1)//ncols + 1
fig, axes = plt.subplots(nrows, ncols, figsize=(12*ncols, 12*nrows))

# Scatter point radiuses
sc_rad = 9
clustersize = 900
clustertextsize = 250

# Color scatter points by their corresponding cluster
# I've written this part of code last year, just copy-pasing  this into this notebook.
cluster_labels = df_clustering['cluster_id'] 
indeces = list(set(df_clustering['cluster_id'].values))
    # SPAGHETTI CODE IM SO SORRY
    # These are the indeces of countries in the created clusters
country_indeces_in_clstrs = [[countries_sc.index(country) for country in list(df_clustering[df_clustering['cluster_id']==idx].index)] for idx in indeces]
colors = cm.nipy_spectral(cluster_labels.astype(float) / len(indeces))
index_colors = cm.nipy_spectral(np.array(indeces)/len(indeces))

for i in range(nrows):
    for j in range(ncols):
        ax = axes[i][j]
        # Select embedding with the correct perplexity
        df_embedded = df_embedded_variation[i*ncols + j]
        x = df_embedded[:,0]
        y = df_embedded[:,1]
        ax.scatter(x, y,
                   color=colors, s=sc_rad**2, alpha=0.7)
        
        # Create the centers for cluster indeces
        centers = np.array([[np.mean(x[elems]), np.mean(y[elems])] for elems in country_indeces_in_clstrs])
        
        # Draw white circles at cluster centers and label them
        ax.scatter(centers[:, 0], centers[:, 1],
                   marker='o', c='white', alpha=0.7,
                   s=clustersize, edgecolor='black')
        for k, c in enumerate(centers):
            ax.scatter(c[0], c[1],
                       marker='${0}$'.format(indeces[k]), alpha=0.9,
                       s=clustertextsize, edgecolor='black',
                       color=np.array(index_colors[k][:-1]))

        ax.set_title('Perplexity = {0}'.format(perplexity[i*ncols + j]), fontsize=axistitlesize, fontweight='bold')
        ax.set_xlabel('Embedding dim. 1', fontsize=axislabelsize, fontweight='bold', labelpad=15)
        ax.set_ylabel('Embedding dim. 2', fontsize=axislabelsize, fontweight='bold', labelpad=15)
        ax.tick_params(axis='both', which='major', labelsize=axisticksize)

fig.suptitle('Fig. 15. Embeddings of a t-SNE performed with two components.\nThe countries are colored by their corresponding cluster.',
             fontsize=axistitlesize+12, y=0.04)
        
plt.show()

### 5./c. Dendorgram

#### Sourced from:
- https://github.com/scikit-learn/scikit-learn/blob/70cf4a676caa2d2dad2e3f6e4478d64bcb0506f7/examples/cluster/plot_hierarchical_clustering_dendrogram.py

In [ ]:
def plot_dendrogram(model):
    """
    Plot dendogram from the results of a clustering. Also mark countries on the figure.
    
    Paramters:
    ----------
    model : `sklearn.cluster.*`
        The clustering model fitted on the input data.
    """
    
    nrows = 1
    ncols = 1
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*30, nrows*45))

    # Children of hierarchical clustering
    children = model.children_

    # Distances between each pair of children
    # Since we don't have this information, we can use a uniform one for plotting
    distance = np.arange(children.shape[0])

    # The number of observations contained in each cluster level
    no_of_observations = np.arange(2, children.shape[0]+2)

    # Create linkage matrix and then plot the dendrogram
    linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, p=30, truncate_mode=None,
               orientation='right', no_labels=False,
               ax=axes)

    # Change ticklabels on Y-axis
    y_ticks_loc = axes.get_yticks(minor=False)
    y_ticks_str = axes.get_yticklabels(minor=False)
    y_ticks_int = []
    for t in y_ticks_str:
        y_ticks_int.append(int(t.get_text()))
    new_y_ticks = np.array(list(df_wf_sc.index))[y_ticks_int]

    # Write countries instead of indeces
    axes.set_yticks(y_ticks_loc)
    axes.set_yticklabels(new_y_ticks)

    axes.tick_params(axis='both', which='major', labelsize=axisticksize)
    
    fig.suptitle('Fig. 16. Dendogram of the agglomerative clustering on the scaled dataset.',
                 fontsize=axistitlesize+12, y=0.1)

    plt.show()

In [ ]:
plot_dendrogram(model=agglomerative)

The coloring should indicate something about the depth of the agglomeration or clusters?? I really don't know, sorry... However the neighbouring countries on this dendrogam correlates pretty well with the observations in the previous task. For example Hungary (in the middle of the graph) was grouped together with the countries said to be the closest to us by the t-SNE analysis. So I can say, we created a robust model and statistics to decide, which countries are the closest to us in WDI.

In [ ]:
# placeholder for the plot
# rip attached image :(

header### Hints:
 - On total you can get 10 points for fully completing all tasks.
 - Decorate your notebook with, questions, explanation etc, make it self contained and understandable!
 - Comments you code when necessary
 - Write functions for repetitive tasks!
 - Use the pandas package for data loading and handling
 - Use matplotlib and seaborn for plotting or bokeh and plotly for interactive investigation
 - Use the scikit learn package for almost everything
 - Use for loops only if it is really necessary!
 - Code sharing is not allowed between student! Sharing code will result in zero points.
 - If you use code found on web, it is OK, but, make its source clear! 